# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [28]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
import requests
import importlib
from IPython.display import Markdown, display, update_display
import scraper
importlib.reload(scraper)  # Reload module to pick up any changes
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [29]:
# Initialize and constants

# Check if Ollama is running
try:
    response = requests.get("http://localhost:11434", timeout=2)
    if response.status_code == 200:
        print("Ollama is running!")
    else:
        print("Ollama server responded but with unexpected status. Please check if Ollama is running properly.")
except requests.exceptions.RequestException:
    print("Ollama is not running. Please open a terminal and run 'ollama serve'")
    print("Then in another terminal, run 'ollama pull llama3.2' to download the model")
    
MODEL = 'llama3.2'
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

Ollama is running!


In [30]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/1

## First step: Have the LLM figure out which links are relevant

### Use a call to the LLM (via Ollama) to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [31]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [32]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [33]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/

In [34]:
def select_relevant_links(url):
    response = ollama.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [35]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page', 'url': 'https://edwarddonner.com/'}]}

In [36]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = ollama.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [37]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling llama3.2
Found 10 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about me page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company overview page',
   'url': 'https://edwarddonner.com/proficient/'},
  {'type': 'curriculum page', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'careers/jobs page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page for company Nebula', 'url': ''},
  {'type': 'company homepage',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn profile page',
   'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile page', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook profile page',
   'url': 'https://www.facebook.com/edward.donner.52'}],
 "I removed all the duplicate links from your list - I also made an educated guess about a career/job section since that information was in place on other sites, including LinkedIn and Edw

In [38]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling llama3.2
Found 8 relevant links


{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'careers page'},
  {'type': 'company page', 'url': 'https://huggingface.co/brand'},
  {'type': 'blog', 'url': 'https://blog.huggingface.co/'},
  {'type': 'github page', 'url': 'https://github.com/huggingface'},
  {'type': 'discord link', 'url': 'https://join.discord.com/huggingface'},
  {'type': 'status page', 'url': 'https://status.huggingface.co/'},
  {'type': 'changelog page',
   'url': 'https://blog.huggingface.co/changelog-2'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to the LLM (via Ollama)

In [39]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [40]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling llama3.2
Found 7 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
nvidia/personaplex-7b-v1
Updated
about 2 hours ago
•
43.9k
•
1.37k
moonshotai/Kimi-K2.5
Updated
about 23 hours ago
•
11k
•
901
Qwen/Qwen3-TTS-12Hz-1.7B-CustomVoice
Updated
6 days ago
•
139k
•
702
microsoft/VibeVoice-ASR
Updated
1 day ago
•
76.7k
•
696
Tongyi-MAI/Z-Image
Updated
about 11 hours ago
•
584
Browse 2M+ models
Spaces
Running
on
Zero
Featured
937
Qwen3-TTS Demo
🎙
937
Transform text into natural-sounding speech with custom voices
Running
on
Zero
MCP
1.87k
Z Image Turbo
🖼
1.87k
Generate stunning AI images from text descriptions i

In [41]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [42]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [43]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling llama3.2
Found 8 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nnvidia/personaplex-7b-v1\nUpdated\nabout 2 hours ago\n•\n43.9k\n•\n1.37k\nmoonshotai/Kimi-K2.5\nUpdated\nabout 23 hours ago\n•\n11k\n•\n901\nQwen/Qwen3-TTS-12Hz-1.7B-CustomVoice\nUpdated\n6 days ago\n•\n139k\n•\n702\nmicrosoft/VibeVoice-ASR\nUpdated\n1 day ago\n•\n76.7k\n•\n696\nTongyi-MAI/Z-Image\nUpdated\nabout 11 hours ago\n•\n584\nBrowse 2M+ models\nSpaces\nRunning\non\nZero\nFeatured\n

In [44]:
def create_brochure(company_name, url):
    response = ollama.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [45]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling llama3.2
Found 7 relevant links


**Hugging Face: Empowering the Future of Artificial Intelligence**

Welcome to Hugging Face, the AI community building the future of machine learning. We are a collaboration platform dedicated to helping experts from around the world work together to create innovative AI applications.

**Our Mission**
We aim to make it easier for professionals to create, discover, and collaborate on ML models, datasets, and applications. Our platform provides unlimited access to public models, datasets, and applications, allowing users to move faster and explore new possibilities.

**Key Features**

* **Model Hub**: Explore 2 million+ pre-trained models across various platforms, including Hugging Face's own Transformers and Diffusers.
* **Datasets**: Browse 500,000+ datasets, complete with details on usage, licensing, and data sources.
* **Spaces**: Host and collaborate on public models, datasets, and applications using our Zero platform.
* **Community**: Join a vibrant community of AI experts, share knowledge, and learn from others.

**Innovative Applications**

* **Text-to-Speech**: Convert text into natural-sounding speech with custom voices
* **AI Image Generation**: Generate stunning AI images from text descriptions in seconds
* **Image Processing**: Adjust camera angles in images using 3D controls or sliders

**Collaboration and Innovation**
At Hugging Face, we believe that the future of AI is built on collaboration. Our platform provides a space for experts to share knowledge, work together, and drive innovation.

**Careers at Hugging Face**
Join our team of dedicated professionals passionate about AI and collaboration. We offer internships, jobs, and partnerships to help you grow in your career.

**Partner with Us**
Hugging Face is committed to empowering the future of AI. Partner with us to explore new applications, technologies, and use cases that can benefit your organization and community.

Let's build a better future for machine learning together.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from Ollama,
with the familiar typewriter animation

In [46]:
def stream_brochure(company_name, url):
    stream = ollama.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    # chunk 단위로 데이터를 받아오고 출력함
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [47]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling llama3.2
Found 7 relevant links


Hugging Face: Building a Community of AI Innovation

**Company Mission**
At Hugging Face, we aim to build a community that collaborates on the future of Artificial Intelligence (AI). Our platform is designed to bring together machine learning practitioners, researchers, and developers to host, discover, and utilize models, datasets, and applications.

**Our Approach**
We believe in open-source innovation. With our HF Open source stack, you can move faster and build more efficient models. Explore all modalities - from text to image, video, audio, or even 3D. Our platform is designed for collaboration, with features like unlimited public models, datasets, and applications.

**What We Offer**

* **2 Million+ Models**: Browse a vast library of pre-trained models for various tasks, including natural language processing (NLP) and computer vision.
* **Flexible Computing**: Run our models on multiple platforms, including Zero, to access high-performance computing capabilities.
* **Community Support**: Join a community of AI practitioners, researchers, and developers who share knowledge, resources, and expertise.

**Our Culture**

At Hugging Face, we value:

*   Open-source collaboration
*   Fast-paced innovation
*   Diversity in team composition
*   Professional growth

**Why Invest in Us?**
By investing in Hugging Face, you'll be supporting the development of cutting-edge AI solutions that benefit various industries and applications. Our platform is constantly evolving, with new features and models being added regularly.

**Careers at Hugging Face**

We're looking for talented individuals to join our team! Check out our job listings and careers page to explore opportunities in AI research, engineering, and software development.

Ready to be a part of the future?

In [48]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling llama3.2
Found 10 relevant links


# Hugging Face: Empowering the Future of Artificial Intelligence

Hugging Face is a pioneering platform that brings together the machine learning community to collaborate on models, datasets, and applications. Our mission is to accelerate the development of artificial intelligence by providing a collaborative and accessible framework for researchers, developers, and businesses.

## Company Culture

At Hugging Face, we value innovation, transparency, and open-source principles. We believe that collaboration and community engagement are essential to driving progress in AI research and development. Our culture is centered around these values, with a focus on:

* Empowering the machine learning community through our platform and resources
* Fostering innovation and experimentation with cutting-edge technologies
* Providing transparent and accessible tools for every developer and researcher
* Promoting diversity, equity, and inclusion in AI development

## Customers and Applications

Hugging Face is used by leading companies and researchers around the world to build and deploy a wide range of applications, including:

* Natural Language Processing (NLP) and language understanding systems
* Computer Vision and image recognition systems
* Audio processing and speech synthesis applications
* Autonomous vehicles and robotics development

Our platform supports over 2 million pre-trained models and has been adopted by some of the world's leading companies, including NVIDIA, Microsoft, and Google.

## Careers and Opportunities

Join our community of innovators and experts in machine learning, natural language processing, computer vision, and more. We offer a range of opportunities for:

* Researchers and developers to collaborate on cutting-edge projects
* Companies looking to integrate AI-powered solutions into their products and services
* Entrepreneurs interested in building and scaling AI-based startups

At Hugging Face, we're passionate about accelerating the development of artificial intelligence and its potential to transform industries and society. Join us today and contribute to shaping the future of AI.

## Key Features and Benefits

* Unlimited public models, datasets, and applications
* Open-source stack with a wide range of tools and resources
* Collaboration platform for researchers, developers, and businesses
* Support for NLP, computer vision, audio processing, and autonomous vehicles development
* Free and easy-to-use API for easy integration into your application

## Who We Are

Hugging Face is a company that puts the collective intelligence of its community at its core. Our mission is to empower every developer and researcher on the planet to build and deploy their own AI models using our platform.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with LLM APIs (via Ollama), and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>